In [2]:
!pip install matplotlib
!pip install seaborn 
!pip install numpy 
!pip install rdata
!pip install pyreadr

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from pathlib import Path
import os
import pickle
import rdata
import pyreadr

In [4]:
#Read in all data
data = pd.read_csv("clim_conflict_for_R.csv")
area = pd.read_csv("croparea_weight.csv", encoding="windows-1252")
ssa = pd.read_csv("gseasSSA.csv")
cltol = pd.read_csv("precip_climatology.csv", index_col=0)
mss_country = pd.read_csv("mss_countries_new.csv", header=0)
mss_country.iloc[:, 0] = mss_country.iloc[:, 0].astype(str)
mss = mss_country.iloc[:, 0].unique().tolist()

In [5]:
#Import the altogether_montly array
altogether_monthly = np.load("altogether_monthly.npy")

In [6]:
#Generate projections of income and regime change to 2030.
#Income projections
ystart = data.loc[data["year_actual"] == 1981, data.columns[9]]
yend = data.loc[data["year_actual"] == 2002, data.columns[9]]

# Annual growth rate
ygro = np.exp(np.log(yend.values / ystart.values) / 21)

# Projections to 2030: using fixed 2% and historical median
median_yend = np.median(yend.dropna())
median_ygro = np.median(ygro[~np.isnan(ygro)])

incchg = np.array([
    median_yend * (1.02) ** 27,                          # Fixed 2% growth
    median_yend * (median_ygro) ** 27                    # Historical median growth
]) - median_yend

# Polity projections
polstart = data.loc[data["year_actual"] == 1981, data.columns[227]]
polend = data.loc[data["year_actual"] == 2002, data.columns[227]]

polchg = polend.values - polstart.values
pol2030 = polend.values + np.median(polchg[~np.isnan(polchg)])
pol2030 = np.clip(pol2030, None, 10)
polchg_final = np.mean(pol2030 - polend.values)

In [7]:
#Make area weights by region
mask = area.iloc[:, 1] != "Cereals,Total +"
for i in area[mask].index:
    # Find the corresponding cereal row in the top 42 rows with same country
    match = area.iloc[:42]
    match_idx = match[match.iloc[:, 0] == area.iloc[i, 0]].index
    if not match_idx.empty:
        area.at[i, area.columns[4]] = area.at[match_idx[0], area.columns[4]]

#Calculate unweighted area sums by region
unweight = []
for j in range(1, 6):  
    cond = (area.iloc[:, 1] == "land area") & (area.iloc[:, 4] == j)
    total = area.loc[cond, area.columns[3]].sum()
    unweight.append(total)

#Normalize
unweight = np.array(unweight)
unweight = unweight / unweight.sum()

In [8]:
#Climate model projections
clim = np.empty((6, 3, 2, 20))  # dims: region, model, variable (temp/precip), projection

#For temperature, averaging over monthly projected changes
k = 0
for i in range(5):       
    for j in range(3):  
        slice_ = altogether_monthly[i + 3, :, j, k, :20]  # (12 months x 20 projections)
        clim[i, j, k, :] = slice_.mean(axis=0)

#For precip, apply percentage changes to baseline climatatology, then compute total change in precip across months 
k = 1
for i in range(5):
    for j in range(3):
        for n in range(20):
            chg = altogether_monthly[i + 3, :, j, k, n] / 100 + 1
            clim[i, j, k, n] = (np.sum(cltol.iloc[i, :] * chg) / np.sum(cltol.iloc[i, :]) - 1) * 100 
for j in range(3):
    for k in range(2):
        for n in range(20):
            clim[5, j, k, n] = np.sum(clim[0:5, j, k, n] * unweight)
#Fixing an error where NaNs appear as -100 values
clim[clim == -100] = np.nan

C:\Users\chris\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:135: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\chris\AppData\Local\Temp\ipykernel_6704\1190004525.py:16: RuntimeWarning: invalid value encountered in divide
  chg = altogether_monthly[i + 3, :, j, k, n] / 100 + 1
